In [ ]:
sig_correct_path = "../sig_correct.txt"
secrets_path = "../secrets.txt"
logfile = "hammer.log"

In [ ]:
def analyze_faulty_sig(sig_correct, sig_faulty):
    # Analyze faulty sig
    sig_field_widths = [('H(m)', 32), ('FORS', 10560)] + [(f'WOTS{i}', 2400) for i in range(8)] + [('msg', 0)]
    i = 0
    for cf, cc in zip(sig_faulty, sig_correct):
        if cf != cc:
            print(f"Bit {i} differs: {cf} != {cc}")
            for field, width in sig_field_widths:
                if i < width:
                    print(f"Bit {i} is in field {field}")
                    break
                # print(f"{i} -= {width}")
                i -= width
            else:
                print(f"Bit {i} is in field msg")
                continue
            break
        i += 1/2

In [ ]:
import glob
import json
import postprocessing as pp

sig_correct = open(sig_correct_path).readline().strip()
#results_files = glob.glob("../results/results_*.json")
results_files = glob.glob("../results/results_2025-03-12_14-30-56.json")

for results_file in results_files:
    j = json.load(open(results_file))
    if 'experiments' not in j:
        print(f'Skipping {results_file}')
        continue
    j = j['experiments']
    for x in j:
        print(x)
        for result in x['results']:
            if 'Ok' in result:
                sig = result['Ok']['victim_result']['String'].strip()
                if sig != sig_correct:
                    print(x['date'])
                    is_exploitable = pp.is_exploitable(sig_correct, sig, secrets_path)
                    print(f"Exploitable: {is_exploitable}")
                    if is_exploitable:
                        analyze_faulty_sig(sig_correct, sig)


In [ ]:
import glob
import pandas as pd
import zipfile


# Get all exploitable_*.zip files
zip_files = glob.glob('results/*.zip')

candidates = []
for zip_filename in zip_files:
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        with zipf.open('sig_correct.txt') as sig_correct_file:
            sig_correct = sig_correct_file.readline().decode('utf-8').strip()
        with zipf.open('sig_faulty.txt') as sig_faulty_file:
            sig_faulty = sig_faulty_file.readline().decode('utf-8').strip()
    exploitable = pp.is_exploitable(sig_correct, sig_faulty, secrets_path)
    common_prefix_length = pp.prefix(sig_correct, sig_faulty)
    candidates.append([zip_filename, common_prefix_length, len(sig_faulty), len(sig_correct), exploitable])
    
candidates = sorted(candidates, key=lambda x: x[1], reverse=True)

dataset = pd.DataFrame(candidates, columns = ['ID', 'common_prefix_length', 'Faulty len', 'Correct len', 'is_exploitable'])
    
print(dataset)